In [ ]:
from typing import Callable, List, Tuple

import pytorch_lightning as pl
import torch
import torch.nn.functional as F
from torch import nn
from torch.distributions.distribution import Distribution
from pytorch_lightning.loggers import TensorBoardLogger
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


Limits exceeded: Not enough units in project to continue executions. Please contact your project admin.


In [2]:
import torch
from torch import nn
import numpy as np

class L(nn.Linear):
    def __init__(self,n):
        super().__init__(n*(n+1)//2,1,bias=False)
        self.n=n
        self.diag_mask=torch.tensor([ (k+1) * (k+2) // 2 - 1 for k in range(n)])
        self.mask2d=torch.zeros((n*(n+1) // 2),dtype=int)
        for i in range(n):
            for j in range(i+1):
                self.mask2d[i*(i+1)//2+j]=i*n+j
    
    def anti_flatten(self):
        n=self.n
        L=torch.zeros((n**2)).to(device)
        L[self.mask2d]=self.weight.to(device) 
        L=torch.reshape(L,(n,n)).to(device)
        return L        
            
    def log_abs_det(self):
        diag=self.weight[0][self.diag_mask]
        la=torch.log(torch.abs(diag))
        lad=torch.sum(la)
        return lad
    
    def forward(self,x):
        Lwt=torch.t(self.anti_flatten())
        return torch.matmul(x,Lwt)
        
    def adj(self,mat):
        Lw=self.anti_flatten()
        Lwt=torch.t(Lw)
        D=torch.matmul(Lwt,torch.matmul(mat,Lw))
        return D
    

In [3]:
from NFconstants import N_nod, beta , N_traj
from NFandist import get_A, get_C
A=(torch.tensor(get_A(N_nod,beta)).float()).to(device)
def set_random_seed(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)

def A_I(model):
    with torch.no_grad():
        A_D=model.adj(A)
        I=(torch.eye(N_nod)).to(device)
    return torch.linalg.matrix_norm(A_D-I)

In [4]:
class Cheatloss(nn.Module):
    def __init__(self):
        super(Cheatloss, self).__init__()

    def forward(self, model,lad):
        A_D=model.adj(A)
        loss=0.5*torch.trace(A_D)-lad
        return loss
CL=Cheatloss()

In [5]:
class Pipeline(pl.LightningModule):
    def __init__(
        self,
        model,
        criterion,
        optimizer_class=torch.optim.Adam,
        optimizer_kwargs={"lr": 0.001,"weight_decay": 0.01}
    ) -> None:
        super().__init__()
        self.model = model
        self.loss = criterion
        self.optimizer_class = optimizer_class
        self.optimizer_kwargs = optimizer_kwargs


    def configure_optimizers(self):
        optimizer = self.optimizer_class(
            self.model.parameters(), **self.optimizer_kwargs
        )
        return optimizer

    def training_step(self, batch, batch_idx):
        z = batch
        x, log_abs_det = self.model(z), self.model.log_abs_det()
        #log_abs_det=self.model.log_abs_det()
        loss = self.loss(x,log_abs_det)
        self.log('train_loss', loss, prog_bar=True)
        
        return loss


    def on_train_epoch_end(self):
        metric=A_I(self.model)
        self.log("metric",metric, prog_bar=True)
        pass

In [6]:
from LOSS import KL_osc
from Data import train_loader

set_random_seed(42)
L_layer=L(N_nod)
pipeline=Pipeline(model=L_layer,criterion=KL_osc, optimizer_class=torch.optim.Adam,
        optimizer_kwargs={"lr": 0.001, "weight_decay": 0.01})
trainer = pl.Trainer(
    max_epochs=1000,
    logger=TensorBoardLogger(save_dir=f"logs/nf"),
    num_sanity_val_steps=0,
)

trainer.fit(model=pipeline, train_dataloaders=train_loader)
torch.save(L_layer.state_dict(), "L_layer_weights.pth")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
2024-01-21 16:57:53.937626: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-21 16:57:55.309223: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type      | Params
------------------------------------
0 | model | L         | 5.0 K 
1 | loss  | KL_with_S | 0     
------------------------------------
5.0 K     Trainable params
0         Non-trainable params
5.0 K     Total params
0.020     Total estimated model params size (MB)
/home/jupyter/.local/lib/python3.10/site-packages/pytorch_lightning/loop

Training: |          | 0/? [00:00<?, ?it/s]

/home/jupyter/.local/lib/python3.10/site-packages/pytorch_lightning/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


In [ ]:
L_layer=L(N_nod)
L_layer.load_state_dict(torch.load("L_layer_weights.pth"))
pipeline=Pipeline(model=L_layer,criterion=KL_osc, optimizer_class=torch.optim.LBFGS,
        optimizer_kwargs={"lr": 0.00001})

trainer = pl.Trainer(
    max_epochs=200,
    logger=TensorBoardLogger(save_dir=f"logs/nf"),
    num_sanity_val_steps=0,
)

trainer.fit(model=pipeline, train_dataloaders=train_loader)
torch.save(L_layer.state_dict(), "L_layer_weights.pth")

In [10]:
L_layer=L(N_nod)
L_layer.load_state_dict(torch.load("L_layer_weights.pth"))
print(L_layer.adj(A))

tensor([[ 9.4951e-01,  8.6635e-03, -7.5204e-03,  ...,  2.5913e-03,
          1.9455e-03,  1.1769e-03],
        [ 8.6637e-03,  9.9896e-01,  1.1440e-03,  ..., -1.1417e-08,
         -8.2279e-09, -2.6673e-09],
        [-7.5205e-03,  1.1440e-03,  9.9896e-01,  ...,  1.3134e-09,
          2.8978e-10,  4.2857e-11],
        ...,
        [ 2.5912e-03, -1.1418e-08,  1.3115e-09,  ...,  1.0000e+00,
         -1.6694e-08,  4.6356e-08],
        [ 1.9452e-03, -8.2257e-09,  2.8788e-10,  ...,  2.7856e-08,
          1.0000e+00,  3.0513e-08],
        [ 1.1774e-03, -2.6683e-09,  4.3626e-11,  ...,  1.8042e-09,
         -3.1326e-08,  1.0000e+00]], device='cuda:0', grad_fn=<MmBackward0>)
